In [0]:
pip install confluent-kafka

Python interpreter will be restarted.
Collecting confluent-kafka
 Downloading confluent_kafka-1.5.0-cp37-cp37m-manylinux1_x86_64.whl (8.1 MB)
Installing collected packages: confluent-kafka
Successfully installed confluent-kafka-1.5.0
Python interpreter will be restarted.

In [0]:
pip install kafka-python

Python interpreter will be restarted.
Collecting kafka-python
 Downloading kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)
Installing collected packages: kafka-python
Successfully installed kafka-python-2.0.2
Python interpreter will be restarted.

In [0]:
%scala
System.setProperty("ssl.ca.location","dbfs:/FileStore/tables/cloudkarafka.ca");

res1: String = dbfs:/FileStore/tables/cloudkarafka.ca

In [0]:
%scala
System.clearProperty("java.security.auth.login.config");
System.clearProperty("sasl.jaas.config");
System.setProperty("java.security.auth.login.config", "dbfs:/FileStore/tables/cloudkarafka.jaas");
System.setProperty("sasl.jaas.config", "dbfs:/FileStore/tables/cloudkarafka.jaas");
println("auth.lang:" + System.getProperty("java.security.auth.login.config"));
println("sasl.jaas:" + System.getProperty("sasl.jaas.config"));

spark.sparkContext.addFile("dbfs:/FileStore/tables/cloudkarafka.jaas")


auth.lang:dbfs:/FileStore/tables/cloudkarafka.jaas
sasl.jaas:dbfs:/FileStore/tables/cloudkarafka.jaas

In [0]:
#https://docs.confluent.io/clients-confluent-kafka-python/current/index.html
from confluent_kafka import Producer

conf = {
  'bootstrap.servers': "omnibus-01.srvs.cloudkafka.com:9094,omnibus-03.srvs.cloudkafka.com:9094,omnibus-02.srvs.cloudkafka.com:9094",
  "client.id": "Kafka-Producer",
  "security.protocol": "SASL_SSL",
  "sasl.mechanism": "SCRAM-SHA-256",
  "sasl.username": "yrmfqh3q",
  "sasl.password": "WfKJEYL_hwNUHVU8laSbj6gXkPIw_xuc"
}
print("conf:", conf)

topic = "yrmfqh3q-test"
filePath = "/FileStore/tables/customers.csv"
read = spark.read.format('csv').options(header='true').options(inferSchema='true').load(filePath)
print(type(read))
read.show()
print("Read done...", read)


#for row in read.rdd.collect(): #doesnt scale on high volumes
#  producer.produce(topic, key="message", value=str(row.asDict()))
#  producer.flush()
#  print("Row Published", row)

def publishToKafka(row):
  print("publishToKafka:", type(row), row)
  
read.foreach(publishToKafka)

from kafka import KafkaProducer

print(type(read))
def publishToKafka2(rows):
  print("publishToKafka2:", type(rows), rows)
  producer = KafkaProducer(
    bootstrap_servers=['omnibus-01.srvs.cloudkafka.com:9094','omnibus-03.srvs.cloudkafka.com:9094','omnibus-02.srvs.cloudkafka.com:9094'],
    client_id= "Kafka-Producer",
    security_protocol="SASL_SSL",
    sasl_mechanism="SCRAM-SHA-256",
    sasl_plain_username="yrmfqh3q",
    sasl_plain_password="WfKJEYL_hwNUHVU8laSbj6gXkPIw_xuc")
  for row in rows:
    print("publishToKafka2::row:", type(row), row)
    #producer.send(topic, {"message", str(row.asDict())} )
    producer.send(topic, key=b'message', value=bytes(str(row.asDict()), "utf-8"))
  producer.flush() #Flush after queuing up all the rows.

read.foreachPartition(publishToKafka2)


conf: {'bootstrap.servers': 'omnibus-01.srvs.cloudkafka.com:9094,omnibus-03.srvs.cloudkafka.com:9094,omnibus-02.srvs.cloudkafka.com:9094', 'client.id': 'Kafka-Producer', 'security.protocol': 'SASL_SSL', 'sasl.mechanism': 'SCRAM-SHA-256', 'sasl.username': 'yrmfqh3q', 'sasl.password': 'WfKJEYL_hwNUHVU8laSbj6gXkPIw_xuc'}
<class 'pyspark.sql.dataframe.DataFrame'>
+---+-----+------+
 id| name|active|
+---+-----+------+
 1|Cust1| 1|
 2|Cust2| 0|
 3|Cust3| 1|
+---+-----+------+

Read done... DataFrame[id: int, name: string, active: int]
<class 'pyspark.sql.dataframe.DataFrame'>